In [ ]:
import numpy as np
import pandas as pd

In [ ]:
books = pd.read_csv('Books.csv')
users = pd.read_csv('Users.csv')
ratings=pd.read_csv('Ratings.csv')

In [ ]:
books.head()


In [ ]:
ratings.head()


Popularity Based Recommender System


In [ ]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [ ]:
ratings_with_name.groupby('Book-Title').count()

In [ ]:
ratings_with_name.groupby('Book-Title').count()['Book-Rating']

In [ ]:
num_rating_df= ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()

In [ ]:
num_rating_df.rename(columns={'Book-Rating':'num-rating'},inplace=True)
num_rating_df

In [ ]:
avg_rating_df= ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()

In [ ]:
avg_rating_df= ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating_df

In [ ]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

In [ ]:
popular_df=popular_df[popular_df['num-rating']>= 250].sort_values('avg_rating',ascending=False)

In [ ]:
popular_df

In [ ]:
popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')

In [ ]:
popular_df=popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num-rating','avg_rating']]

In [ ]:
popular_df

Collaborative Filtering Based Recommender System


In [ ]:
ratings_with_name

In [ ]:
ratings_with_name.groupby('User-ID').count()['Book-Rating']

In [ ]:
ratings_with_name

In [ ]:
x=ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
x[x]

In [ ]:
ratings_with_name

In [ ]:
padhe_likhe_users=x[x].index

In [ ]:
filtered_rating=ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [ ]:
y=filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = y[y].index

In [ ]:
filtered_rating['Book-Title'].isin(famous_books)

In [ ]:
final_ratings=filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [ ]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [ ]:
pt.fillna(0,inplace=True)

In [ ]:
pt

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_scores=cosine_similarity(pt)

In [ ]:
similarity_scores.shape

(656, 656)

In [ ]:
def recommend(book_name):
  index =np.where(pt.index == book_name)[0][0]
  similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x :x[1],reverse=True)[1:6]

  for i in similar_items:
    print(pt.index[i[0]])

In [ ]:
def recommend(book_name):
  index =np.where(pt.index == book_name)[0][0]
  similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x :x[1],reverse=True)[1:6]

  data=[]

  for i in similar_items:
    item=[]
    temp_df = books[books['Book-Title'] == pt.index[i[0]]]
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
    item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))

    data.append(item)

  return data

In [ ]:
recommend('Message in a Bottle')

[['Nights in Rodanthe',
  'Nicholas Sparks',
  'http://images.amazon.com/images/P/0446531332.01.MZZZZZZZ.jpg'],
 ['A Walk to Remember',
  'Nicholas Sparks',
  'http://images.amazon.com/images/P/0446608955.01.MZZZZZZZ.jpg'],
 ['The Mulberry Tree',
  'Jude Deveraux',
  'http://images.amazon.com/images/P/0743437640.01.MZZZZZZZ.jpg'],
 ["River's End",
  'Nora Roberts',
  'http://images.amazon.com/images/P/0515127833.01.MZZZZZZZ.jpg'],
 ['All I Really Need to Know',
  'ROBERT FULGHUM',
  'http://images.amazon.com/images/P/080410526X.01.MZZZZZZZ.jpg']]

In [ ]:
np.where(pt.index == '1984')[0][0]

0

This gave us a list of similarity scores of  book at index 0 with itself and with others.

In [ ]:
list(enumerate(similarity_scores[0]))

In [ ]:
sorted(list(enumerate(similarity_scores[0])),key=lambda x :[1],reverse=True)[1:6]

[(1, 0.10976162494155216),
 (2, 0.013319165645513949),
 (3, 0.0),
 (4, 0.058053798706210094),
 (5, 0.03009549402798291)]

In [ ]:
recommend('1984')

In [ ]:
popular_df.columns

Index(['Book-Title', 'Book-Author', 'Image-URL-M', 'num-rating', 'avg_rating'], dtype='object')

In [ ]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [ ]:
list(popular_df['Book-Title'].values)

In [ ]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))